## Dependencies

In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np

data_path = Path('./data/EchoNet-Dynamic/EchoNet-Dynamic/')
file_list_path = os.path.join(data_path,'FileList.csv')
volume_tracings_path = os.path.join(data_path,'VolumeTracings.csv')
videos_path = os.path.join(data_path,'Videos')
os.chdir('../')
%pwd

'c:\\Projects\\python\\echoframe'

In [2]:
file_list = pd.read_csv(filepath_or_buffer=file_list_path)
volume_tracings = pd.read_csv(filepath_or_buffer=volume_tracings_path)


In [3]:
volume_tracings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425010 entries, 0 to 425009
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   FileName  425010 non-null  object 
 1   X1        425010 non-null  float64
 2   Y1        425010 non-null  float64
 3   X2        425010 non-null  float64
 4   Y2        425010 non-null  float64
 5   Frame     425010 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 19.5+ MB


## Dataset preprocessing

### Handling missing files and outliers

In [4]:
file_list['FileName'] = file_list['FileName'].apply(
    lambda x: x if x.endswith('.avi') else x + '.avi'
)
vt_filenames = set(volume_tracings['FileName'])
fl_filenames = set(file_list['FileName'])

missing_files = list(fl_filenames - vt_filenames)
extra_files = list(vt_filenames - fl_filenames)

print(f'Missing files: \n{missing_files}\n\nExtra files: \n{extra_files}')
redacted_files = missing_files+extra_files

redacted_files

Missing files: 
['0X6C435C1B417FDE8A.avi', '0X2DC68261CBCC04AE.avi', '0X5515B0BD077BE68A.avi', '0X234005774F4CB5CD.avi', '0X5DD5283AC43CCDD1.avi', '0X35291BE9AB90FB89.avi']

Extra files: 
['0X4F8859C8AB4DA9CB.avi']


['0X6C435C1B417FDE8A.avi',
 '0X2DC68261CBCC04AE.avi',
 '0X5515B0BD077BE68A.avi',
 '0X234005774F4CB5CD.avi',
 '0X5DD5283AC43CCDD1.avi',
 '0X35291BE9AB90FB89.avi',
 '0X4F8859C8AB4DA9CB.avi']

In [5]:
file_list = file_list[~file_list['FileName'].isin(redacted_files)]
volume_tracings = volume_tracings[~volume_tracings['FileName'].isin(redacted_files)]

### Train test and val splits

In [6]:
train_df = file_list[file_list['Split'] == 'TRAIN']
val_df   = file_list[file_list['Split'] == 'VAL']
test_df  = file_list[file_list['Split'] == 'TEST']